In [1]:
import library.link_preprocessor as lp
import library.query_builder as qb
import library.web_crawler as wc
import library.credibility_analysis as cra
import library.content_analysis as ca
import library.dataframe_preprocessor as dp
import library.doc_similarity as ds
import pandas as pd
import numpy as np

from datetime import date

In [2]:
top_hits_df = pd.read_excel('AMS_tophits.xlsx')
top_hits_df

,URL,Date,Source,Source Type,Target Search,Adverse Type,Credibility Score,Recency,Heading,Content,Adversity Score,Google_rank,Date_rank
0,https://www.jungewelt.de/artikel/440573.finanz...,2022-12-12,www.jungewelt.de,Newspaper/Magazine,Rabobank,CAMS,1,1,Geldwäsche und Kartellbildung,Geldwäsche und Kartellbildung »Die Staatsanwal...,0.9992,22,4
1,https://www.jungewelt.de/artikel/440573.finanz...,2022-12-12,www.jungewelt.de,Newspaper/Magazine,Rabobank,ESG,1,1,Geldwäsche und Kartellbildung,Geldwäsche und Kartellbildung »Die Staatsanwal...,0.9992,230,1
2,https://www.fxstreet.com/news/usd-jpy-eyes-est...,2022-12-12,www.fxstreet.com,Newspaper/Magazine,Rabobank,CAMS,1,1,USD/JPY eyes establishment above 137.00 as anx...,USD/JPY eyes establishment above 137.00 as anx...,0.9249,113,2
3,https://www.brecorder.com/news/40214013,2022-12-12,www.brecorder.com,Newspaper/Magazine,Rabobank,CAMS,1,1,"Soybeans slip further from 3-month top, wheat ...","Soybeans slip further from 3-month top, wheat ...",0.6652,112,3
4,https://www.geenstijl.nl/5168219/snackbarverbo...,2022-12-12,www.geenstijl.nl,Newspaper/Magazine,Rabobank,CAMS,1,1,"Jep, ook wij gebruiken cookies","Jep, ook wij gebruiken cookies Sorry, niet onz...",0.1531,154,5
5,https://road.cc/content/blog/davide-rebellin-m...,2022-12-12,road.cc,Newspaper/Magazine,Rabobank,CAMS,1,1,Davide Rebellin made me fall in love with prof...,Davide Rebellin made me fall in love with prof...,0.0540,115,0
6,https://www.volkskrant.nl/nieuws-achtergrond/i...,2022-12-12,www.volkskrant.nl,Newspaper/Magazine,Rabobank,CAMS,1,1,NaN,NaN,0.0000,100,6
7,https://www.wielerflits.nl/nieuws/patrick-lefe...,2022-12-11,www.wielerflits.nl,Newspaper/Magazine,Rabobank,ESG,1,1,Patrick Lefevere streng: “Julian Alaphilippe i...,Patrick Lefevere streng: “Julian Alaphilippe i...,0.9989,207,7
8,https://akkrum.net/actueel/toestemming-opvang-...,2022-12-11,akkrum.net,Newspaper/Magazine,Rabobank,ESG,1,1,Actueel,Actueel De opvang van Oekraïners in Akkrum is ...,0.8538,197,8
9,https://www.duic.nl/algemeen/rechter-legt-acti...,2022-12-10,www.duic.nl,Newspaper/Magazine,Rabobank,ESG,1,1,\n\n\n\n\n,"Dat komt mede door het forse politieoptreden, ...",0.9835,187,9


In [3]:
top_hits_df_notsocialmedia = top_hits_df.loc[top_hits_df['Source Type'] != 'Social Media']
top_hits_df_notsocialmedia = top_hits_df_notsocialmedia[top_hits_df_notsocialmedia['Source Type'].notna()]
top_hits_df_notsocialmedia = top_hits_df_notsocialmedia[top_hits_df_notsocialmedia['Content'].notna()]

top_hits_df_notsocialmedia['Preprocessed Content'] = top_hits_df_notsocialmedia['Content'].apply(dp.text_process)
doc_similarity_df = ds.pairwise_doc_similariy(list(top_hits_df_notsocialmedia['URL']), list(top_hits_df_notsocialmedia['Content']))

avg_doc_similarity_scores = ds.compute_avg_similarity(np.array(doc_similarity_df))

top_hits_df_notsocialmedia['Reliability Score'] = avg_doc_similarity_scores
top_hits_df_notsocialmedia = top_hits_df_notsocialmedia[['URL', 'Reliability Score']]

top_hits_df = pd.merge(top_hits_df, top_hits_df_notsocialmedia, on = 'URL', how = 'outer')
top_hits_df.drop_duplicates(inplace = True)

In [4]:
top_hits_df

,URL,Date,Source,Source Type,Target Search,Adverse Type,Credibility Score,Recency,Heading,Content,Adversity Score,Google_rank,Date_rank,Reliability Score
0,https://www.jungewelt.de/artikel/440573.finanz...,2022-12-12,www.jungewelt.de,Newspaper/Magazine,Rabobank,CAMS,1,1,Geldwäsche und Kartellbildung,Geldwäsche und Kartellbildung »Die Staatsanwal...,0.9992,22,4,0.347583
2,https://www.jungewelt.de/artikel/440573.finanz...,2022-12-12,www.jungewelt.de,Newspaper/Magazine,Rabobank,ESG,1,1,Geldwäsche und Kartellbildung,Geldwäsche und Kartellbildung »Die Staatsanwal...,0.9992,230,1,0.347583
4,https://www.fxstreet.com/news/usd-jpy-eyes-est...,2022-12-12,www.fxstreet.com,Newspaper/Magazine,Rabobank,CAMS,1,1,USD/JPY eyes establishment above 137.00 as anx...,USD/JPY eyes establishment above 137.00 as anx...,0.9249,113,2,0.293268
5,https://www.brecorder.com/news/40214013,2022-12-12,www.brecorder.com,Newspaper/Magazine,Rabobank,CAMS,1,1,"Soybeans slip further from 3-month top, wheat ...","Soybeans slip further from 3-month top, wheat ...",0.6652,112,3,0.239621
6,https://www.geenstijl.nl/5168219/snackbarverbo...,2022-12-12,www.geenstijl.nl,Newspaper/Magazine,Rabobank,CAMS,1,1,"Jep, ook wij gebruiken cookies","Jep, ook wij gebruiken cookies Sorry, niet onz...",0.1531,154,5,0.168751
7,https://road.cc/content/blog/davide-rebellin-m...,2022-12-12,road.cc,Newspaper/Magazine,Rabobank,CAMS,1,1,Davide Rebellin made me fall in love with prof...,Davide Rebellin made me fall in love with prof...,0.0540,115,0,0.082484
8,https://www.volkskrant.nl/nieuws-achtergrond/i...,2022-12-12,www.volkskrant.nl,Newspaper/Magazine,Rabobank,CAMS,1,1,NaN,NaN,0.0000,100,6,NaN
9,https://www.wielerflits.nl/nieuws/patrick-lefe...,2022-12-11,www.wielerflits.nl,Newspaper/Magazine,Rabobank,ESG,1,1,Patrick Lefevere streng: “Julian Alaphilippe i...,Patrick Lefevere streng: “Julian Alaphilippe i...,0.9989,207,7,0.187901
10,https://akkrum.net/actueel/toestemming-opvang-...,2022-12-11,akkrum.net,Newspaper/Magazine,Rabobank,ESG,1,1,Actueel,Actueel De opvang van Oekraïners in Akkrum is ...,0.8538,197,8,0.190074
11,https://www.duic.nl/algemeen/rechter-legt-acti...,2022-12-10,www.duic.nl,Newspaper/Magazine,Rabobank,ESG,1,1,\n\n\n\n\n,"Dat komt mede door het forse politieoptreden, ...",0.9835,187,9,0.254363


In [5]:
top_hits_df.to_excel('AMS_tophits_ReliabilityAnalysis.xlsx')